In [1]:
import pandas as pd

In [2]:
MODEL_COLUMN = "model"
BASELINE="BEVfusion"
CORRUPTIONS = ["beamsreducing", "brightness", "dark", "fog", "missingcamera", "motionblur", "pointsreducing", "snow", "spatialmisalignment", "temporalmisalignment"]
NICE_NAMES = {
    "beamsreducing": "Beams Red.",
    "brightness": "Brightness",
    "dark": "Darkness",
    "fog":"Fog",
    "missingcamera": "Missing Cam.",
    "motionblur": "Motion Blur",
    "pointsreducing": "Points Red.",
    "snow": "Snow",
    "spatialmisalignment": "Spatial Mis.",
    "temporalmisalignment": "Temporal Mis."
}
MODELS_TO_LINKS = {
    "CMT": "https://github.com/junjie18/CMT",
    "DeepInteraction": "https://github.com/fudan-zvg/DeepInteraction",
    "TransFusion": "https://github.com/XuyangBai/TransFusion",
    "Sparsefusion": "https://github.com/yichen928/SparseFusion",
    "BEVfusion": "https://github.com/mit-han-lab/bevfusion",
    "IS-Fusion": "https://github.com/yinjunbo/IS-Fusion",
    "UniBEV_cat": "https://github.com/tudelft-iv/UniBEV",
    "UniBEV_avg": "https://github.com/tudelft-iv/UniBEV",
    "UniBEV_cnw": "https://github.com/tudelft-iv/UniBEV",
    "MEFormer": "https://github.com/hanchaa/MEFormer",
    "MEFormer w/o PME": "https://github.com/hanchaa/MEFormer",
    "MoME": "https://github.com/konyul/MoME"
}
MODELS = list(MODELS_TO_LINKS.keys())
MODELS_TO_LINKS_MD = {k: f"[{k}]({v})" for k, v in MODELS_TO_LINKS.items()}
SEVERITY_LEVELS = ["1", "2", "3"]

In [3]:
df = pd.read_csv("../evaluation/evaluations_nds.csv")

In [4]:
df = df.dropna()

In [5]:
df

,model,clean,temporalmisalignment_s1,temporalmisalignment_s2,temporalmisalignment_s3,spatialmisalignment_s1,spatialmisalignment_s2,spatialmisalignment_s3,pointsreducing_s1,pointsreducing_s2,...,dark_s3,brightness_s1,brightness_s2,brightness_s3,beamsreducing_s1,beamsreducing_s2,beamsreducing_s3,snow_s1,snow_s2,snow_s3
0,CMT,0.7290,0.6572,0.5680,0.4988,0.6462,0.5860,0.5367,0.7012,0.6834,...,0.6786,0.7038,0.6852,0.6605,0.6826,0.5873,0.4497,0.6483,0.6190,0.5554
1,DeepInteraction,0.6909,0.6139,0.5237,0.4544,0.5863,0.5014,0.4267,0.6522,0.6242,...,0.6254,0.6785,0.6719,0.6574,0.6162,0.4571,0.2839,0.5526,0.5311,0.4896
2,TransFusion,0.7084,0.6306,0.5437,0.4766,0.5708,0.4761,0.4095,0.6554,0.6190,...,0.6978,0.7056,0.7041,0.7007,0.6222,0.4591,0.2636,0.5447,0.5353,0.5087
3,Sparsefusion,0.7315,0.6523,0.5622,0.4901,0.6061,0.5149,0.4461,0.6881,0.6578,...,0.6978,0.7205,0.7146,0.7039,0.6593,0.5139,0.3380,0.5859,0.5704,0.5337
4,BEVfusion,0.7144,0.6341,0.5471,0.4085,0.5903,0.4943,0.4266,0.6684,0.6379,...,0.6861,0.6967,0.6913,0.6851,0.6396,0.4899,0.3199,0.5782,0.5572,0.5233
5,IS-Fusion,0.7365,0.6537,0.5609,0.4892,0.6221,0.5214,0.4359,0.6827,0.6489,...,0.6925,0.7198,0.7091,0.6917,0.6694,0.5068,0.3266,0.5619,0.5463,0.5115
6,UniBEV_cat,0.6782,0.6097,0.5283,0.4644,0.5813,0.5126,0.4390,0.6470,0.6263,...,0.6461,0.6717,0.6629,0.6493,0.6141,0.5101,0.4199,0.5811,0.5292,0.4390
7,UniBEV_avg,0.6842,0.6147,0.5330,0.4677,0.5801,0.5117,0.4619,0.6530,0.6313,...,0.6501,0.6757,0.6658,0.6537,0.6183,0.5151,0.4264,0.5849,0.5269,0.4323
8,UniBEV_cnw,0.6853,0.6122,0.5292,0.4646,0.5794,0.5065,0.4492,0.6514,0.6295,...,0.6518,0.6764,0.6670,0.6524,0.6181,0.5061,0.3981,0.5798,0.5229,0.4224
9,MEFormer,0.7385,0.6659,0.5762,0.5013,0.6494,0.5866,0.5274,0.7078,0.6863,...,0.6919,0.7143,0.6982,0.6788,0.6885,0.5837,0.4201,0.6622,0.6288,0.5565


## Resistance Ability (RA)

In [6]:
for c in CORRUPTIONS:
    df[f"RA_{c}_s1"] = df[c+"_s1"] / df['clean']
    df[f"RA_{c}_s2"] = df[c+"_s2"] / df['clean']
    df[f"RA_{c}_s3"] = df[c+"_s3"] / df['clean']
    df[f"RA_{c}"] = (df[f"RA_{c}_s1"] + df[f"RA_{c}_s2"] + df[f"RA_{c}_s3"]) / 3
    
df["mRA"] = df[[f"RA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [7]:
df[[MODEL_COLUMN]+[f"RA_{c}_s1" for c in CORRUPTIONS] + [f"RA_{c}_s2" for c in CORRUPTIONS] + [f"RA_{c}_s3" for c in CORRUPTIONS] + ["mRA"]]

,model,RA_beamsreducing_s1,RA_brightness_s1,RA_dark_s1,RA_fog_s1,RA_missingcamera_s1,RA_motionblur_s1,RA_pointsreducing_s1,RA_snow_s1,RA_spatialmisalignment_s1,...,RA_brightness_s3,RA_dark_s3,RA_fog_s3,RA_missingcamera_s3,RA_motionblur_s3,RA_pointsreducing_s3,RA_snow_s3,RA_spatialmisalignment_s3,RA_temporalmisalignment_s3,mRA
0,CMT,0.936351,0.965432,0.967215,0.956653,0.984911,0.944582,0.961866,0.889300,0.886420,...,0.906036,0.930864,0.567078,0.961454,0.734842,0.875171,0.761866,0.736214,0.684225,0.864856
1,DeepInteraction,0.891880,0.982052,0.954407,0.837748,0.905051,0.948039,0.943986,0.799826,0.848603,...,0.951513,0.905196,0.210161,0.779129,0.714865,0.798668,0.708641,0.617600,0.657693,0.794944
2,TransFusion,0.878317,0.996047,0.991671,0.918408,0.990542,0.949605,0.925184,0.768916,0.805759,...,0.989130,0.985037,0.498165,0.980519,0.697911,0.752541,0.718097,0.578063,0.672784,0.823970
3,Sparsefusion,0.901299,0.984962,0.973616,0.924402,0.972932,0.956528,0.940670,0.800957,0.828571,...,0.962269,0.953930,0.520984,0.934518,0.734655,0.798086,0.729597,0.609843,0.669993,0.833575
4,BEVfusion,0.895297,0.975224,0.978443,0.917553,0.985162,0.960386,0.935610,0.809351,0.826288,...,0.958987,0.960386,0.501680,0.962066,0.766377,0.787794,0.732503,0.597144,0.571809,0.829685
5,IS-Fusion,0.908893,0.977325,0.964155,0.909301,0.972709,0.956687,0.926952,0.762933,0.844671,...,0.939172,0.940258,0.535370,0.933741,0.787916,0.772709,0.694501,0.591853,0.664223,0.825522
6,UniBEV_cat,0.905485,0.990416,0.979799,0.935712,0.982896,0.934975,0.953996,0.856827,0.857122,...,0.957387,0.952669,0.565025,0.956503,0.666618,0.851519,0.647302,0.647302,0.684754,0.847130
7,UniBEV_avg,0.903683,0.987577,0.980123,0.927507,0.977930,0.930576,0.954399,0.854867,0.847852,...,0.955422,0.950161,0.536977,0.950745,0.667787,0.850775,0.631833,0.675095,0.683572,0.843983
8,UniBEV_cnw,0.901941,0.987013,0.980009,0.928498,0.973734,0.926310,0.950533,0.846053,0.845469,...,0.951992,0.951116,0.532322,0.941194,0.645994,0.846637,0.616372,0.655479,0.677951,0.836801
9,MEFormer,0.932295,0.967231,0.968720,0.944617,0.965606,0.949086,0.958429,0.896682,0.879350,...,0.919160,0.936899,0.555992,0.914963,0.715640,0.867569,0.753555,0.714150,0.678808,0.856005


In [8]:
for c in CORRUPTIONS:
    df[[MODEL_COLUMN]+[f"RA_{c}_s1", f"RA_{c}_s2", f"RA_{c}_s3"]].T.reset_index(drop=True).to_csv(f"../evaluation/ra_results_{c}.csv", index=False, header=False)

In [9]:
df[[MODEL_COLUMN]+[f"RA_{c}_s1" for c in CORRUPTIONS] + [f"RA_{c}_s2" for c in CORRUPTIONS] + [f"RA_{c}_s3" for c in CORRUPTIONS] + ["mRA"]].to_csv("../evaluation/ra_results.csv", index=False)

# Relative Resistance Ability (RRA)

In [10]:
for c in CORRUPTIONS:
    non_baseline = df.loc[df[MODEL_COLUMN] != BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1)
    baseline = df.loc[df[MODEL_COLUMN] == BASELINE, f"{c}_s1":f"{c}_s3"].sum(axis=1).iloc[0]
    df[f"RRA_{c}"] = (((non_baseline / baseline) - 1)) * 100
    df.loc[df[MODEL_COLUMN] == BASELINE, f"RRA_{c}"] = 0
    
df["mRRA"] = df[[f"RRA_{c}" for c in CORRUPTIONS]].mean(axis=1)

In [11]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS] + ["mRRA"]]

,model,clean,RRA_beamsreducing,RRA_brightness,RRA_dark,RRA_fog,RRA_missingcamera,RRA_motionblur,RRA_pointsreducing,RRA_snow,RRA_spatialmisalignment,RRA_temporalmisalignment,mRRA
0,CMT,0.7290,18.642197,-1.138392,-0.096334,9.398263,2.041012,-0.840834,8.212509,9.887261,17.052673,8.448135,7.160649
1,DeepInteraction,0.6909,-6.361253,-3.149872,-7.215452,-25.037221,-16.385588,-7.077023,-2.188219,-5.148610,0.211752,0.144681,-7.220680
2,TransFusion,0.7084,-7.209880,1.799238,1.146380,-0.552109,0.340169,-5.411524,-3.295704,-4.220172,-3.626257,3.849783,-1.718008
3,Sparsefusion,0.7315,4.263833,3.178814,1.820722,4.429280,0.297049,0.280278,3.242202,1.887020,3.699047,7.227779,3.032602
4,BEVfusion,0.7144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,IS-Fusion,0.7365,3.684283,2.291255,1.266798,3.889578,0.919893,3.993963,1.690653,-2.351239,4.512970,7.177455,2.707561
6,UniBEV_cat,0.6782,6.533738,-4.302735,-5.279129,-0.198511,-5.437907,-12.504716,-0.979081,-6.595527,1.435945,0.798893,-2.652903
7,UniBEV_avg,0.6842,7.616945,-3.757658,-4.595154,-1.228288,-5.169605,-11.777071,-0.144455,-6.909025,2.812335,1.616657,-2.153532
8,UniBEV_cnw,0.6853,5.029667,-3.728715,-4.383219,-1.104218,-5.749329,-13.119172,-0.428013,-8.054501,1.581525,1.025351,-2.893062
9,MEFormer,0.7385,16.758659,0.877912,1.555802,8.585608,-0.167689,-0.307228,8.865229,11.382408,16.688724,9.668491,7.390792


In [12]:
df[[MODEL_COLUMN]+["clean"]+[f"RRA_{c}" for c in CORRUPTIONS]].T.to_csv("../evaluation/rra_nds_results.csv", header=False)